## 資料分析期末 ：決策樹

### 二手車資料

In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
cars=pd.read_excel("/Users/jen/Downloads/raw_used_car_data.xlsx")

In [35]:
cars.head(5)

,make,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price,type
0,Smart,200.0,45.0,used,NaN,semi-automatic,gasoline,127000.0,1850.0,1
1,Smart,199.0,40.0,used,1.0,semi-automatic,gasoline,118000.0,1990.0,1
2,Smart,152.0,70.0,used,NaN,semi-automatic,diesel,220000.0,2300.0,1
3,Citroen,133.0,100.0,used,NaN,semi-automatic,diesel,182000.0,2300.0,1
4,Smart,165.0,70.0,used,NaN,semi-automatic,gasoline,120000.0,2500.0,1


In [36]:
cars.shape

(1048575, 10)

In [37]:
car=cars.dropna(axis=0,how='any')
car.shape

(21418, 10)

In [38]:
x=pd.DataFrame(car[['months_old','kms','num_owners']])
#print('target name:'+ str(iris['target_names']))
y=pd.DataFrame(car['type'],columns=['type'])
iris_data= pd.concat([x,y],axis=1)
#iris_data=iris_data[['sepal length (cm)','petal length (cm)','target']]
#iris_data=iris_data[iris_data['type'].isin([0,1])]
#iris_data.tail()
iris_data.head()

,months_old,kms,num_owners,type
1,199.0,118000.0,1.0,1
8,165.0,96600.0,1.0,1
12,174.0,91000.0,1.0,1
33,129.0,190000.0,1.0,1
34,118.0,137000.0,1.0,1


In [39]:
iris_data['type']=iris_data['type'].astype('int')

In [40]:
from sklearn.model_selection import train_test_split

In [41]:
X_train,X_test,y_train,y_test=train_test_split(iris_data[['months_old','kms','num_owners']],iris_data['type'],test_size=0.3,random_state=0)

In [42]:
from sklearn.tree import DecisionTreeClassifier

In [43]:
clf = DecisionTreeClassifier(criterion = 'entropy', random_state=0)

In [44]:
clf.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [45]:
clf.predict(X_test)

array([2, 4, 6, ..., 3, 5, 2])

In [46]:
y_test.values

array([7, 2, 6, ..., 7, 3, 2])

In [47]:
error = 0
for i, v in enumerate(clf.predict(X_test)):
    if v!= y_test.values[i]:
        #print(i,v)
        error+=1
print(error)

4427


In [48]:
clf.score(X_test,y_test.values)

0.3110799875505758

## 嘗試找出影響最大的變數，簡化決策樹

In [49]:
import statsmodels.formula.api as sm

In [50]:
result = sm.ols(formula="price ~ months_old+ kms +power + C(make) +sale_type+num_owners+gear_type+fuel_type ", data=car).fit()
print (result.summary())
#make	months_old	power	sale_type	num_owners	gear_type	fuel_type	kms

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     652.8
Date:                Thu, 13 Jun 2019   Prob (F-statistic):               0.00
Time:                        10:33:53   Log-Likelihood:            -2.2997e+05
No. Observations:               21418   AIC:                         4.600e+05
Df Residuals:                   21366   BIC:                         4.605e+05
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [52]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for col in car.columns:
    car[col] = labelencoder.fit_transform(car[col])
car.head()

/Users/jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,make,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price,type
1,30,198,5,4,0,2,5,6591,45,0
8,30,164,7,4,0,2,5,5967,85,0
12,30,173,5,4,0,2,5,5733,92,0
33,6,128,42,4,0,2,2,7637,156,0
34,30,117,24,4,0,2,5,6973,156,0


In [53]:
car.shape

(21418, 10)

In [54]:
y = car['type']
X = car.drop('type', axis=1)
X = X.drop('price', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.8)
columns = X_train.columns

/Users/jen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [55]:
from sklearn.preprocessing import StandardScaler
ss_X = StandardScaler()
ss_y = StandardScaler()
X_train = ss_X.fit_transform(X_train)
X_test = ss_X.transform(X_test)

/Users/jen/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/jen/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/Users/jen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """


In [57]:
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [58]:
y_prob = model_tree.predict_proba(X_test)[:,1]
y_pred = np.where(y_prob > 0.5, 1, 0)
model_tree.score(X_test, y_pred)

0.2852474323062558

In [60]:
import pydotplus

dot_data = tree.export_graphviz(model_tree, out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf('iris.pdf')

True

In [27]:
from sklearn.tree import export_graphviz
import graphviz
data_feature_name=car[:0]
export_graphviz(model_tree, out_file="mytree.pdf")
with open("mytree.pdf") as f:
    dot_graph = f.read()
#graphviz.Source(dot_graph)

NameError: name 'model_tree' is not defined

In [28]:
data_feature_name=car[:0]
data_feature_name

,make,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price,type


In [29]:
cars.head()

,make,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price,type
0,Smart,200.0,45.0,used,NaN,semi-automatic,gasoline,127000.0,1850.0,1
1,Smart,199.0,40.0,used,1.0,semi-automatic,gasoline,118000.0,1990.0,1
2,Smart,152.0,70.0,used,NaN,semi-automatic,diesel,220000.0,2300.0,1
3,Citroen,133.0,100.0,used,NaN,semi-automatic,diesel,182000.0,2300.0,1
4,Smart,165.0,70.0,used,NaN,semi-automatic,gasoline,120000.0,2500.0,1


In [30]:
cars=cars.fillna(value=0)
cars.head()

,make,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price,type
0,Smart,200.0,45.0,used,0.0,semi-automatic,gasoline,127000.0,1850.0,1
1,Smart,199.0,40.0,used,1.0,semi-automatic,gasoline,118000.0,1990.0,1
2,Smart,152.0,70.0,used,0.0,semi-automatic,diesel,220000.0,2300.0,1
3,Citroen,133.0,100.0,used,0.0,semi-automatic,diesel,182000.0,2300.0,1
4,Smart,165.0,70.0,used,0.0,semi-automatic,gasoline,120000.0,2500.0,1


In [31]:
from sklearn.datasets import load_iris
from sklearn import tree
import sys
import os 
import pydotplus
dot_data = tree.export_graphviz(model_tree, out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("iris.pdf")

NameError: name 'model_tree' is not defined

In [45]:
cars.fillna(0)

,make,months_old,power,sale_type,num_owners,gear_type,fuel_type,kms,price,type
0,Smart,200.0,45.0,used,0.0,semi-automatic,gasoline,127000.0,1850.0,1
1,Smart,199.0,40.0,used,1.0,semi-automatic,gasoline,118000.0,1990.0,1
2,Smart,152.0,70.0,used,0.0,semi-automatic,diesel,220000.0,2300.0,1
3,Citroen,133.0,100.0,used,0.0,semi-automatic,diesel,182000.0,2300.0,1
4,Smart,165.0,70.0,used,0.0,semi-automatic,gasoline,120000.0,2500.0,1
5,Smart,152.0,80.0,used,0.0,semi-automatic,gasoline,119000.0,2500.0,1
6,Smart,156.0,70.0,used,0.0,semi-automatic,diesel,210000.0,2600.0,1
7,Smart,125.0,45.0,used,0.0,semi-automatic,gasoline,165000.0,2700.0,1
8,Smart,165.0,45.0,used,1.0,semi-automatic,gasoline,96600.0,2700.0,1
9,Smart,133.0,45.0,used,0.0,semi-automatic,gasoline,144000.0,2800.0,1


In [49]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
for col in cars.columns:
    cars[col] = labelencoder.fit_transform(cars[col])
cars.head()

TypeError: '<' not supported between instances of 'str' and 'int'

In [50]:
y = cars['type']
X = cars.drop('type', axis=1)
X = X.drop('price', axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.8)
columns = X_train.columns

/Users/jen/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [51]:
from sklearn.preprocessing import StandardScaler
ss_X = StandardScaler()
ss_y = StandardScaler()
X_train = ss_X.fit_transform(X_train)
X_test = ss_X.transform(X_test)

ValueError: could not convert string to float: 'Volkswagen'